In [53]:
#!pip install fredapi
!pip install tqdm

In [56]:
import os
import time
from tqdm.notebook import tqdm_notebook as tqdm
from datetime import datetime

In [4]:
SECRET_PATH = '../secrets'
DATA_PATH = '../data'

In [17]:
from fredapi import Fred
import pandas as pd

# Initialize the FRED API client with your API key
fred = Fred(api_key=fred_api_key)

# Define the FRED series IDs for the data you want to download
series_ids = [
    'UNRATE',  # Unemployment rate
    'GDP',     # Gross domestic product
    'CPALTT01USM657N',  # Consumer Price Index for All Urban Consumers: All Items
]

# Download the FRED data for the specified series IDs
data = {}
for series_id in series_ids:
    series = fred.get_series(series_id)
    data[series_id] = series

# Combine the data into a single dataframe and save it to a CSV file
df = pd.DataFrame(data)

try:
    df.to_csv(os.path.join(DATA_PATH, formatted_date_time, 'fred_data.csv'), index=False)
except FileNotFoundError as e:
    os.mkdir(os.path.join(DATA_PATH, formatted_date_time))
    df.to_csv(os.path.join(DATA_PATH, formatted_date_time, 'fred_data.csv'), index=False)


In [59]:
import os
from datetime import datetime, timedelta
import pandas as pd
from fredapi import Fred


def get_fred_data(api_key, series_ids):
    csv_filename = 'fred_data.csv'
    csv_filename_path = os.path.join(DATA_PATH, csv_filename)

    # Check if cached data exists and is less than a month old
    if os.path.exists(csv_filename):
        file_age = datetime.now() - datetime.fromtimestamp(os.path.getmtime(csv_filename))
        if file_age < timedelta(days=30):
            # Read data from CSV file
            df = pd.read_csv(csv_filename)
            return df[df['series_id'].isin(series_ids)]

    # Read data from FRED API
    fred = Fred(api_key=api_key)
    df = pd.DataFrame()
    for series_id in tqdm(series_ids):
        time.sleep(0.5)
        try:
            data = fred.get_series(series_id)
            df = df.append(pd.DataFrame({'series_id': [series_id] * len(data), 'date': data.index, 'value': data.values}))
        except Exception as e:
            print(series_id + str(e))
    try:
        df.to_csv(csv_filename_path, index=False)
    except FileNotFoundError as e:
        os.mkdir(csv_filename)
        df.to_csv(csv_filename_path, index=False)

    return df


import os
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_stock_data(api_key, symbol, interval='1min', outputsize='compact'):
    csv_filename = f'{symbol}.csv'
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&outputsize={outputsize}&apikey={api_key}'

    # Check if cached data exists and is less than a day old
    if os.path.exists(csv_filename):
        file_age = datetime.now() - datetime.fromtimestamp(os.path.getmtime(csv_filename))
        if file_age < timedelta(days=1):
            # Read data from CSV file
            df = pd.read_csv(csv_filename)
            return df

    # Fetch data from API
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()['Time Series (1min)']
        df = pd.DataFrame.from_dict(data, orient='index')
        df.index = pd.to_datetime(df.index)
        df.sort_index(inplace=True)
        df.to_csv(csv_filename)  # Cache data to CSV file
        return df
    else:
        print(f'Error fetching data for {symbol}')
        return None

In [60]:
def main():

    with open(os.path.join(SECRET_PATH,'fred_api_key.txt')) as f:
        fred_api_key = f.read()

    with open(os.path.join(SECRET_PATH,'alpha_vantage_api_key.txt')) as f:
        av_api_key = f.read()

    series_ids_df = pd.read_csv('../metadata/fred_series_ids.txt')
    
    fred_df = get_fred_data(fred_api_key, list(series_ids_df['Series ID']))
    
    return fred_df

fred_df = main()

  0%|          | 0/30 [00:00<?, ?it/s]

WTIBad Request.  The series does not exist.
DTWEXBPABad Request.  The series does not exist.
SP500DIVBad Request.  The series does not exist.
CPILTT01USM657NBad Request.  The series does not exist.
DSPIC96TBad Request.  The series does not exist.


In [61]:
fred_df

,series_id,date,value
0,GDP,1946-01-01,NaN
1,GDP,1946-04-01,NaN
2,GDP,1946-07-01,NaN
3,GDP,1946-10-01,NaN
4,GDP,1947-01-01,243.164
...,...,...,...
12026,DGS30,2023-03-22,3.680
12027,DGS30,2023-03-23,3.660
12028,DGS30,2023-03-24,3.640
12029,DGS30,2023-03-27,3.770


In [62]:
fred_df.pivot(index='date', columns = 'series_id', values = 'value')

series_id,A191RL1Q225SBEA,BAMLH0A0HYM2,CPIAUCSL,CPILFESL,DCOILWTICO,DGS10,DGS30,DSPIC96,DTWEXB,DTWEXEMEGS,...,M2SL,MORTGAGE30US,PAYEMS,PCE,PCETRIM12M159SFRBDAL,SP500,T10Y3M,UNRATE,VIXCLS,WTISPLC
date,,,,,,,,,,,,,,,,,,,,,
1919-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-23,NaN,5.10,NaN,NaN,69.77,3.38,3.66,NaN,NaN,126.6667,...,NaN,6.42,NaN,NaN,NaN,3948.72,-1.35,NaN,22.61,NaN
2023-03-24,NaN,5.22,NaN,NaN,69.22,3.38,3.64,NaN,NaN,127.1277,...,NaN,NaN,NaN,NaN,NaN,3970.99,-1.36,NaN,21.74,NaN
2023-03-27,NaN,5.03,NaN,NaN,72.87,3.53,3.77,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3977.53,-1.38,NaN,20.60,NaN
